In [492]:
import numpy as np
import pandas as pd
import matplotlib as plt
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import get_close_matches

data = pd.read_csv('top10K-TMDB-movies.csv')
df = pd.DataFrame(data)
# find and remove mssing data

new_data = df.dropna()
# new_data = new_data['title']

# if df.isnull().sum().any() :
#     print('found')
# else:
#     print('none')

# if new_data.isnull().sum().any() :
#     print('found')
# else:
#     print('none')
# print( new_data.isnull().sum())

new_data['final_tags'] = new_data['genre'] + new_data['overview']

final_data = new_data.drop(['genre', 'original_language', 'overview', 'popularity',
       'vote_average', 'vote_count','release_date',],axis=1)

tfvectorize = TfidfVectorizer(stop_words='english')
tf_matrix = tfvectorize.fit_transform(final_data['final_tags'])

cosine_func = cosine_similarity(tf_matrix)


print(cosine_func[0])


C:\Users\sakthi\AppData\Local\Temp\ipykernel_19712\1602084241.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['final_tags'] = new_data['genre'] + new_data['overview']


[1.         0.01078192 0.00778483 ... 0.04233725 0.02887264 0.01411831]


In [493]:
def recommend_movie(title,n):

    idx = final_data[final_data['title'] == title].index[0]
    # print(idx)

    cosine_sim = list(enumerate(cosine_func[idx]))
    # print(cosine_sim)
    # print(list(enumerate(['sakthi','ram'])))
    
    cosine_sim = sorted(cosine_sim, key=lambda x : x[1], reverse=True )
    print(cosine_sim)

    movie_incidies = cosine_sim[1:n+1]
    # print(movie_incidies)

    movie_id = [movie[0] for movie in movie_incidies]
    # print(movie_id)

    movie = final_data['title'].iloc[movie_id]
    

    return list(movie)
    

recommend_movie('The Shawshank Redemption',10)
# new_data
# new_data


[(0, np.float64(1.0)), (6154, np.float64(0.13758088948539066)), (2991, np.float64(0.11303706823686718)), (4328, np.float64(0.11008479729141732)), (791, np.float64(0.10827221569506684)), (4085, np.float64(0.10535424646047781)), (698, np.float64(0.10117585760496783)), (4067, np.float64(0.09903230058738414)), (6275, np.float64(0.09749900961137499)), (617, np.float64(0.09655890413719077)), (6559, np.float64(0.09502405865153227)), (4488, np.float64(0.09187399818085534)), (1970, np.float64(0.09079325029784897)), (2927, np.float64(0.0895161588474268)), (87, np.float64(0.08926260818225645)), (8634, np.float64(0.08454971682222398)), (6910, np.float64(0.08451296946433637)), (3648, np.float64(0.08416984234888066)), (9000, np.float64(0.08386144406586783)), (5180, np.float64(0.0831655588555763)), (1009, np.float64(0.08210046329528109)), (7867, np.float64(0.08198690209832346)), (3182, np.float64(0.0814642238268273)), (2120, np.float64(0.08117404722353634)), (6400, np.float64(0.07866339487146912)), (

['In Hell',
 'Brubaker',
 'Demolition',
 'The Chorus',
 'One Shot',
 'Cool Hand Luke',
 'Fatal Attraction',
 'The 40 Year Old Virgin',
 'Toy Story 3',
 'No Escape']

In [494]:
from sklearn.neighbors import NearestNeighbors


model = NearestNeighbors(n_neighbors=11, metric='cosine')
model.fit(tf_matrix)

indices = pd.Series(final_data.index, final_data['title'])


def reccommend_movie_KN(title,n):
    idx = indices[title]
    # print(idx)

    incidies_list = model.kneighbors(tf_matrix[idx], n_neighbors=11)
    # print(incidies_list[1])
    result = []

    for movie in incidies_list[1]:
        # print(incidies_list[1])
        movie_list = list( final_data['title'].iloc[movie])
        movie = movie_list[1:n+1]
        return movie




In [497]:
import requests
from time import sleep  # Optional: be polite to the API

API_KEY = 'a01b8453d45c3f5806c7397aa4addae5'
BASE_URL = 'https://api.themoviedb.org/3/search/movie'

# recommended = ['Interstellar', 'Inception', 'The Matrix', 'Titanic', 'Avatar',
#                'The Dark Knight', 'Fight Club', 'Forrest Gump', 'Gladiator', 'Up']

movie_list = reccommend_movie_KN("Spider-Man",5)

def get_movies(title):
    params = {
        'api_key': API_KEY,
        'query': title
    }

    res = requests.get(BASE_URL, params= params)
    data = res.json()
    # print(data['results'])

    if data['results']:
        movie = data['results'][0]
        return {
            'title': movie['title'],
            'poster_url': f"https://image.tmdb.org/t/p/w500{movie['poster_path']}" if movie.get('poster_path') else None,
            'rating': movie['vote_average'],
            # 'overview': movie['overview'],
            'release_date': movie['release_date']
        }
    return {'title': title, 'error': 'Not found'}

# get_movies(title='')
total_movies_list =[]

for m in movie_list:
    movie = get_movies(m)
    total_movies_list.append(movie)
    sleep(0.2)

for movie in total_movies_list:
    print(movie)




{'title': 'Spider-Man 3', 'poster_url': 'https://image.tmdb.org/t/p/w500/qFmwhVUoUSXjkKRmca5yGDEXBIj.jpg', 'rating': 6.439, 'release_date': '2007-05-01'}
{'title': 'Arachnophobia', 'poster_url': 'https://image.tmdb.org/t/p/w500/lHjpwv1Yv2RcGcTQb4Lc1cKw1y9.jpg', 'rating': 6.433, 'release_date': '1990-07-20'}
{'title': 'Spider-Man: Homecoming', 'poster_url': 'https://image.tmdb.org/t/p/w500/c24sv2weTHPsmDa7jEMN0m2P3RT.jpg', 'rating': 7.331, 'release_date': '2017-07-05'}
{'title': 'Spider-Man: Into the Spider-Verse', 'poster_url': 'https://image.tmdb.org/t/p/w500/iiZZdoQBEYBv6id8su7ImL0oCbD.jpg', 'rating': 8.398, 'release_date': '2018-12-06'}
{'title': 'The Amazing Spider-Man', 'poster_url': 'https://image.tmdb.org/t/p/w500/jexoNYnPd6vVrmygwF6QZmWPFdu.jpg', 'rating': 6.716, 'release_date': '2012-06-23'}


{'title': 'Interstellar', 'poster_url': 'https://image.tmdb.org/t/p/w500/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg', 'rating': 8.456, 'overview': 'The adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage.', 'release_date': '2014-11-05'}
{'title': 'Inception', 'poster_url': 'https://image.tmdb.org/t/p/w500/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg', 'rating': 8.369, 'overview': 'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.', 'release_date': '2010-07-15'}
{'title': 'The Matrix', 'poster_url': 'https://image.tmdb.org/t/p/w500/p96dm7sCMn4VYAStA6siNz30G1r.jpg', 'rating': 8.227, 'overview': 'Set in the 22nd century, The Matrix tells the st